# Training Plots

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *
from load_datasets import load_and_prep_dataset

import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.io import loadmat
import copy
import pandas as pd
import seaborn as sns

2024-07-11 18:32:58.399344: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis_gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train Loop and IMP loop

In [2]:
# modified train loop to also work with sparse networks (such that pruned weights remain frozen at 0.0)

def train_loop_sparse(train, test, model, num_epochs=5):
    
    # hyperparameters
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
    loss_function= tf.keras.losses.CategoricalCrossentropy()
    
    # initializing training statistics
    train_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')
    test_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')
    train_losses = tf.keras.metrics.CategoricalCrossentropy(name='train_losses')
    test_losses = tf.keras.metrics.CategoricalCrossentropy(name='test_losses')
    train_acc = []
    test_acc = []
    train_l =[]
    test_l = []

    for epoch in tqdm(range(num_epochs), leave=False, desc="training epochs"):
        
        #train step
        for x, t in train:
            with tf.GradientTape() as tape:
                pred = model(x)
                loss = loss_function(t, pred)
                train_losses.update_state(t, pred)
                train_accuracy.update_state(tf.argmax(t,1), tf.argmax(pred,1))
            gradients = tape.gradient(loss, model.trainable_variables)
            
            # set gradients to zero for all pruned weights
            new_gradients = []
            for gradient_matrix, variables in zip(gradients, model.trainable_variables):
                gradient_matrix = tf.where(variables == 0.0, 0.0, gradient_matrix)
                new_gradients.append(gradient_matrix)
            optimizer.apply_gradients(zip(new_gradients, model.trainable_variables))
            
        # test step
        for x, t in test:
            pred = model(x)
            test_accuracy.update_state(tf.argmax(t,1), tf.argmax(pred,1))
            test_losses.update_state(t, pred)
        
        # updataing training statistics
        train_acc.append(train_accuracy.result().numpy())
        test_acc.append(test_accuracy.result().numpy())
        train_l.append(train_losses.result().numpy())
        test_l.append(test_losses.result().numpy())
        train_accuracy.reset_state()
        test_accuracy.reset_state()
        train_losses.reset_state()
        test_losses.reset_state()
        
    # collecting losses in a dictionary
    losses = {"test loss":test_l , "training loss":train_l , "test accuracy":test_acc , "training accuracy":train_acc}
    losses = pd.DataFrame.from_dict(losses, orient='columns')
    
    return  losses

## Training

- Winning Tickets
- Random Tickets
- Control Models

In [3]:
def train_control(train_dataset, test_dataset, nr):

    all_losses = pd.DataFrame()

    for i in range(nr):

        control_model = CNN2Model()
        control_losses = train_loop_sparse(train_dataset, test_dataset, control_model)
        control_losses["model_nr"] = i
        control_losses["Sparsity"] = "original"
        control_losses["epochs"] = control_losses.index

        all_losses = pd.concat([all_losses, control_losses], axis=0)
    
    return all_losses
    


In [4]:
def train_wts(train_dataset, test_dataset, dataset, nrs):

    all_losses = pd.DataFrame()

    for i in nrs:

        wt_model = CNN2Model()
        wt_model(list(train_dataset)[0][0])
        wt_model.load_weights(f"1b WTs/h5_format/WT_{dataset}_IMP_{i}.h5")


        wt_losses = train_loop_sparse(train_dataset, test_dataset, wt_model)
        wt_losses["model_nr"] = i
        wt_losses["Sparsity"] = "winning ticket"
        wt_losses["epochs"] = wt_losses.index

        all_losses = pd.concat([all_losses, wt_losses], axis=0)
    
    return all_losses

In [5]:
def train_random_sparse(train_dataset, test_dataset, nrs):

    all_losses = pd.DataFrame()

    for i in nrs:

        random_sparse_model = CNN2Model()
        random_sparse_model(list(train_dataset)[0][0])
        random_sparse_model.load_weights(f"1b WTs/h5_format/RSN_{i}.h5")


        random_sparse_losses = train_loop_sparse(train_dataset, test_dataset, random_sparse_model)
        random_sparse_losses["model_nr"] = i
        random_sparse_losses["Sparsity"] = "winning ticket"
        random_sparse_losses["epochs"] = random_sparse_losses.index

        all_losses = pd.concat([all_losses, random_sparse_losses], axis=0)
    
    return all_losses

In [6]:
for dataset in ["CIFAR", "CINIC", "SVHN"]:

    nr = 15

    train_dataset, test_dataset = load_and_prep_dataset(dataset, batch_size=60, shuffle_size=512)

    control_stats = train_control(train_dataset, test_dataset, nr)
    control_stats["dataset"]=dataset
    #display(control_stats)
    control_stats.to_csv(f'6b Training statistics/{dataset}_original_training_stats.csv', index=False)

    wts_stats = train_wts(train_dataset, test_dataset, dataset, range(nr))
    wts_stats["dataset"]=dataset
    #display(wts_stats)
    wts_stats.to_csv(f'6b Training statistics/{dataset}_WT_training_stats.csv', index=False)

    random_stats = train_random_sparse(train_dataset, test_dataset, range(nr))
    random_stats["dataset"]=dataset
    #display(random_stats)
    random_stats.to_csv(f'6b Training statistics/{dataset}_RSN_training_stats.csv', index=False)

    coll_stats = pd.concat([control_stats,wts_stats,random_stats], axis =0)
    coll_stats.to_csv(f'6b Training statistics/{dataset}_collective_training_stats.csv', index=False)

2024-07-11 18:33:04.043675: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-11 18:33:04.091762: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-11 18:33:04.091958: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis_gpu/lib/python3.12/site-packages/keras/src/layers/pooling/base_pooling.py:23: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)
training epochs:   0%|          | 0/5 [00:00<?, ?it/s]2024-07-11 18:33:04.620136: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-07-11 18:33:48.049641: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-11 18:33:50.043377: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
training epochs:  20%|██        | 1/5 [00:45<03:02, 45.63s/it]2024-07-11 18:34:29.500881: W tensorflow/core/framework/local_rendezv

,test loss,training loss,test accuracy,training accuracy,model_nr,Sparsity,epochs,dataset
0,1.183304,1.416766,0.582485,0.492027,0,original,0,CIFAR
1,1.087838,1.027313,0.620809,0.636071,0,original,1,CIFAR
2,0.928950,0.849676,0.676547,0.702918,0,original,2,CIFAR
3,0.916773,0.712222,0.682934,0.751538,0,original,3,CIFAR
4,0.907775,0.590212,0.692914,0.796721,0,original,4,CIFAR
0,1.162265,1.400360,0.585679,0.499960,1,original,0,CIFAR
1,0.993697,1.014099,0.652994,0.642806,1,original,1,CIFAR
2,0.948484,0.829177,0.667914,0.710612,1,original,2,CIFAR
3,0.896181,0.685706,0.692814,0.763568,1,original,3,CIFAR
4,0.906286,0.554762,0.703094,0.809991,1,original,4,CIFAR


/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis_gpu/lib/python3.12/site-packages/keras/src/layers/pooling/base_pooling.py:23: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)
2024-07-11 18:40:00.706698: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
training epochs:   0%|          | 0/5 [00:00<?, ?it/s]2024-07-11 18:40:40.739345: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-11 18:40:42.396261: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
training epochs:  20%|██        | 1/5 [00:41<02:45, 41.42s/it]2024-07-11 18:41:22.253764: W ten

,test loss,training loss,test accuracy,training accuracy,model_nr,Sparsity,epochs,dataset
0,0.927561,0.993802,0.675798,0.679236,0,winning ticket,0,CIFAR
1,0.943138,0.490927,0.691317,0.846104,0,winning ticket,1,CIFAR
2,1.058515,0.264348,0.692415,0.927821,0,winning ticket,2,CIFAR
3,1.164108,0.143362,0.701347,0.967789,0,winning ticket,3,CIFAR
4,1.351352,0.079342,0.695160,0.986412,0,winning ticket,4,CIFAR
0,0.920658,0.997408,0.680689,0.677358,1,winning ticket,0,CIFAR
1,0.923947,0.501646,0.693214,0.841408,1,winning ticket,1,CIFAR
2,1.014489,0.275688,0.701597,0.924304,1,winning ticket,2,CIFAR
3,1.145585,0.150120,0.701497,0.964931,1,winning ticket,3,CIFAR
4,1.331540,0.085146,0.696158,0.984734,1,winning ticket,4,CIFAR


TypeError: 'int' object is not iterable